In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import os

# Constants for the game
EMPTY = 0
PLAYER_X = 1
PLAYER_O = 2
BOARD_SIZE = 8
WIN_LENGTH = 4
DEPTH = 3

class Problem:
    def __init__(self):
        # Initialize the board with zeros (empty cells)
        self.board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
        self.current_player = PLAYER_X

    def print_board(self):
        # Print the column indices
        print(" ", end=" ")
        for col in range(BOARD_SIZE):
            print(f"{col:>3}", end=" ")
        print()

        # Define symbols for empty, player X, and player O
        symbols = {EMPTY: ".", PLAYER_X: "X", PLAYER_O: "O"}
        # Print each row with its index and the corresponding cell symbols
        for i, row in enumerate(self.board):
            print(i, end=" ")
            for cell in row:
                print(f"{symbols[cell]:>3}", end=" ")
            print()

    def check_winner(self, player):
        # Check all possible lines on the board for a win condition
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if j <= BOARD_SIZE - WIN_LENGTH and all(self.board[i][j + k] == player for k in range(WIN_LENGTH)):
                    return True
                if i <= BOARD_SIZE - WIN_LENGTH and all(self.board[i + k][j] == player for k in range(WIN_LENGTH)):
                    return True
                if j <= BOARD_SIZE - WIN_LENGTH and i <= BOARD_SIZE - WIN_LENGTH and all(self.board[i + k][j + k] == player for k in range(WIN_LENGTH)):
                    return True
                if j >= WIN_LENGTH - 1 and i <= BOARD_SIZE - WIN_LENGTH and all(self.board[i + k][j - k] == player for k in range(WIN_LENGTH)):
                    return True
        return False

    def is_draw(self):
        # Check if there are any empty cells left on the board
        return not any(EMPTY in row for row in self.board)

    def get_empty_positions(self):
        # Get a list of all empty positions on the board
        return [(i, j) for i in range(BOARD_SIZE) for j in range(BOARD_SIZE) if self.board[i][j] == EMPTY]

    def evaluate_line(self, line, player):
        # Evaluate a line of cells for a given player
        count = np.count_nonzero(line == player)
        if count == WIN_LENGTH:
            return 10000
        elif count == WIN_LENGTH - 1 and np.count_nonzero(line == EMPTY) == 1:
            return 5000
        elif count == WIN_LENGTH - 2 and np.count_nonzero(line == EMPTY) == 2:
            return 1000
        return 0

    def evaluate_board(self):
        # Evaluate the board and return a score
        score = 0

        # Check all horizontal lines
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE - WIN_LENGTH + 1):
                horizontal_line = self.board[i, j:j + WIN_LENGTH]
                score += self.evaluate_line(horizontal_line, PLAYER_X)
                score -= self.evaluate_line(horizontal_line, PLAYER_O)

        # Check all vertical lines
        for i in range(BOARD_SIZE - WIN_LENGTH + 1):
            for j in range(BOARD_SIZE):
                vertical_line = self.board[i:i + WIN_LENGTH, j]
                score += self.evaluate_line(vertical_line, PLAYER_X)
                score -= self.evaluate_line(vertical_line, PLAYER_O)

        # Check all diagonal lines
        for i in range(BOARD_SIZE - WIN_LENGTH + 1):
            for j in range(BOARD_SIZE - WIN_LENGTH + 1):
                diagonal_line1 = np.array([self.board[i + k, j + k] for k in range(WIN_LENGTH)])
                diagonal_line2 = np.array([self.board[i + k, j + WIN_LENGTH - 1 - k] for k in range(WIN_LENGTH)])
                score += self.evaluate_line(diagonal_line1, PLAYER_X)
                score -= self.evaluate_line(diagonal_line1, PLAYER_O)
                score += self.evaluate_line(diagonal_line2, PLAYER_X)
                score -= self.evaluate_line(diagonal_line2, PLAYER_O)

        return score

    def get_player_move(self):
        # Get the player's move from input
        while True:
            try:
                move = input("Enter your move (row and column): ").split()
                if len(move) != 2:
                    raise ValueError("Invalid input")
                row, col = map(int, move)
                if row < 0 or row >= BOARD_SIZE or col < 0 or col >= BOARD_SIZE:
                    raise ValueError("Move out of board")
                if self.board[row][col] != EMPTY:
                    raise ValueError("Cell already taken")
                return row, col
            except ValueError as e:
                print(e)

class SearchStrategy:
    def __init__(self, problem):
        self.problem = problem

    def maximize(self, depth, alpha, beta):
        # Maximizing player (AI) tries to get the highest score
        max_eval = -float('inf')
        best_move = None
        for move in self.sort_moves(self.problem.get_empty_positions()):
            self.problem.board[move[0]][move[1]] = self.problem.current_player
            eval, _ = self.alpha_beta_search(depth - 1, alpha, beta, False)
            self.problem.board[move[0]][move[1]] = EMPTY
            if eval > max_eval:
                max_eval = eval
                best_move = move
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move

    def minimize(self, depth, alpha, beta):
        # Minimizing player (opponent) tries to get the lowest score
        min_eval = float('inf')
        best_move = None
        for move in self.sort_moves(self.problem.get_empty_positions()):
            self.problem.board[move[0]][move[1]] = self.problem.current_player
            eval, _ = self.alpha_beta_search(depth - 1, alpha, beta, True)
            self.problem.board[move[0]][move[1]] = EMPTY
            if eval < min_eval:
                min_eval = eval
                best_move = move
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

    def alpha_beta_search(self, depth, alpha, beta, maximizing_player):
        # Alpha-beta pruning algorithm to search the game tree
        if depth == 0 or self.problem.check_winner(PLAYER_X) or self.problem.check_winner(PLAYER_O) or self.problem.is_draw():
            return self.problem.evaluate_board(), None

        if maximizing_player:
            return self.maximize(depth, alpha, beta)
        else:
            return self.minimize(depth, alpha, beta)

    def sort_moves(self, moves):
        # Sort moves based on their distance to the center of the board
        center = BOARD_SIZE // 2
        return sorted(moves, key=lambda move: (move[0] - center)**2 + (move[1] - center)**2)

import os

def main():
    # Main function to run the game
    game_board = Problem()
    search_strategy = SearchStrategy(game_board)

    while True:
        os.system('cls' if os.name == 'nt' else 'clear')  # Clear console screen

        game_board.print_board()
        if game_board.current_player == PLAYER_X:
            row, col = game_board.get_player_move()
        else:
            _, move = search_strategy.alpha_beta_search(DEPTH, -float('inf'), float('inf'), False)
            row, col = move
            print(f"AI chooses move: {row} {col}")

        game_board.board[row][col] = game_board.current_player

        if game_board.check_winner(game_board.current_player):
            os.system('cls' if os.name == 'nt' else 'clear')  # Clear console screen
            game_board.print_board()
            print(f"Player {'X' if game_board.current_player == PLAYER_X else 'O'} wins!")
            break

        if game_board.is_draw():
            os.system('cls' if os.name == 'nt' else 'clear')  # Clear console screen
            game_board.print_board()
            print("The game is a draw!")
            break

        game_board.current_player = PLAYER_O if game_board.current_player == PLAYER_X else PLAYER_X

if __name__ == "__main__":
    main()


    0   1   2   3   4   5   6   7 
0   .   .   .   .   .   .   .   . 
1   .   .   .   .   .   .   .   . 
2   .   .   .   .   .   .   .   . 
3   .   .   .   .   .   .   .   . 
4   .   .   .   .   .   .   .   . 
5   .   .   .   .   .   .   .   . 
6   .   .   .   .   .   .   .   . 
7   .   .   .   .   .   .   .   . 
Enter your move (row and column): 7, 8
invalid literal for int() with base 10: '7,'
Enter your move (row and column): 7
Invalid input
Enter your move (row and column): 7 8
Move out of board
Enter your move (row and column): 6 7
    0   1   2   3   4   5   6   7 
0   .   .   .   .   .   .   .   . 
1   .   .   .   .   .   .   .   . 
2   .   .   .   .   .   .   .   . 
3   .   .   .   .   .   .   .   . 
4   .   .   .   .   .   .   .   . 
5   .   .   .   .   .   .   .   . 
6   .   .   .   .   .   .   .   X 
7   .   .   .   .   .   .   .   . 
AI chooses move: 4 4
    0   1   2   3   4   5   6   7 
0   .   .   .   .   .   .   .   . 
1   .   .   .   .   .   .   .   . 
2   .   .   .   